In [18]:
#using Pkg
#Pkg.activate("D:\\Science\\Code\\Julia\\envr")
using StaticArrays, DifferentialEquations, DynamicalSystems, JLD, CairoMakie, BenchmarkTools

In [2]:
function TM(u, p, t)

    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    
    # u[1] - E; u[2] - x; u[3] - y
    # α - 1, τ - 2, τD - 3, τy - 4, J - 5, xthr - 6, ythr - 7, U0 - 8, ΔU0 - 9, β - 10, I0 -11

    du1 = (-u[1] + p[1] * log( 1.0 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1.0 - u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2])
    return SA[du1, du2, du3]
end;

In [3]:
t = 500.0; tstep = 0.001
trange = range(0.0, t, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [4]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305;

In [5]:
I0range = range( -1.40, -1.709, length = 201 )
U0range = range( 0.3,  0.47, length = 201)

0.3:0.00085:0.47

In [6]:
path = "C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\2М Карта спектра Ляпуновских показателей\\200x200";
filenameLS = "\\LSs_200x200.jld"
filenameu0s = "\\u0s_200x200.jld"
pathLS = path*filenameLS
pathu0s = path*filenameu0s;

In [7]:
Λs = load(pathLS)["data"]
u0s = load(pathu0s)["data"];

In [8]:
modes = zeros(201, 201);

In [9]:
function check_chaos_mode(idx_I0, idx_U0)
    chaos = 0
    if Λs[idx_I0, idx_U0, 1] >= 0.1
            chaos = 3
    end
    return chaos
end
function local_min(tr)
    
    localmin = Float64[]
    placeintime = Float64[]
    
    for idx in range(2, length(tr) - 1, step = 1)
        if tr[idx-1] > tr[idx] < tr[idx+1]
            push!(localmin, tr[idx])
            push!(placeintime, trange[idx])
        end
    end
    return localmin, placeintime
end
function traj(idx_I0, idx_U0)
    
    E, x, y = u0s[idx_I0, idx_U0, :]
    u0 = SA[E, x, y]
    I0_ = I0range[idx_I0]
    U0_ = U0range[idx_U0]
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_]
    ds = ContinuousDynamicalSystem(TM, u0, p)
    tr = trajectory(ds, t; diffeq = integ_set)
    
    return tr[:, 1]
end
function detect_mode(idx_I0, idx_U0)
    
    mode = check_chaos_mode(idx_I0, idx_U0)
    if mode == 0
        tr = traj(idx_I0, idx_U0)
        lcmin, _ = local_min(tr)
        if abs(maximum(lcmin) - minimum(lcmin)) > 2
            mode = 2
        else
            mode = 1
        end
    end
    return mode
end

detect_mode (generic function with 1 method)

In [10]:
function output(idx_U0, U0, idx_I0, I0, mode)
    println("index U0: $idx_U0; index I0: $idx_I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("Mode: $mode");flush(stdout)
end
function separate()
    println("");flush(stdout)
    println("----------------");flush(stdout)
end

separate (generic function with 1 method)

In [11]:
idxI0 = 90; idxU0 = 1
E, x, y = u0s[idxI0, idxU0, :]
u0 = SA[E, x, y]
p = SA[α, τ, τD, τy, J, xthr, ythr, U0range[idxU0], ΔU0, β, I0range[idxI0]];

In [12]:
Λs[idxI0, idxU0, :]

3-element Vector{Float64}:
  0.001878280258316038
 -1.536370413919485
 -2.767665641148319

In [13]:
ds = ContinuousDynamicalSystem(TM, u0, p)

3-dimensional continuous dynamical system
 state:       [14.7168, 0.691488, 0.432762]
 rule f:      TM
 in-place?    false
 jacobian:    ForwardDiff
 parameters:  [1.58, 0.013, 0.08, 3.3, 3.07, …, 0.3, 0.305, 0.3, -1.5375]

In [53]:
tr = trajectory(ds, t; diffeq = integ_set);
lcmin, timemin = local_min(tr[:, 1]);

In [32]:
@benchmark trajectory(ds, t; save_idxs = 1, diffeq = integ_set)

BenchmarkTools.Trial: 30 samples with 1 evaluation.
 Range (min … max):  170.349 ms … 177.270 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     171.070 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   171.582 ms ±   1.470 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄▁▁▁█ ▄▁       ▁        ▄                                      
  █████▆██▆▆▁▁▁▆▁█▁▁▆▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  170 ms           Histogram: frequency by time          177 ms <

 Memory estimate: 395.94 KiB, allocs estimate: 82.

In [15]:
detect_mode(idxI0, idxU0)

2

In [26]:
abs(maximum(lcmin) - minimum(lcmin))

2.194377640694831

In [55]:
tstart, tend = 1, 1000
f = Figure(resolution = (1000, 300))
ax = Axis(f[1, 1])
lines!(trange[tstart:tend], tr[tstart:tend])
#scatter!(timemin[1:23], lcmin[1:23])
f

LoadError: `Makie.convert_arguments` for the plot type Lines{Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, Dataset{3, Float64}}} and its conversion trait PointBased() was unsuccessful.

The signature that could not be converted was:
::Vector{Float32}, ::Dataset{3, Float64}

Makie needs to convert all plot input arguments to types that can be consumed by the backends (typically Arrays with Float32 elements).
You can define a method for `Makie.convert_arguments` (a type recipe) for these types or their supertypes to make this set of arguments convertible (See http://docs.makie.org/stable/documentation/recipes/index.html).

Alternatively, you can define `Makie.convert_single_argument` for single arguments which have types that are unknown to Makie but which can be converted to known types and fed back to the conversion pipeline.


In [ ]:
for (idx_U0, U0) in enumerate(U0range)
    for (idx_I0, I0) in enumerate(I0range)
        
        modes[idx_I0,idx_U0] =  detect_mode(idx_I0, idx_U0)
        output(idx_U0, U0, idx_I0, I0, modes[idx_I0,idx_U0])
        separate()
        
    end
end   

In [ ]:
I0range = range( -1.40, -1.709, length = 201 )
U0range = range( 0.3,  0.47, length = 201)

f = Figure()
ax1 = Axis(f[1, 1], xlabel = L"I0",ylabel = L"U0", xlabelsize = 50, ylabelsize = 50,
            xticklabelsize = 35, yticklabelsize = 35,
            xgridvisible  = false, ygridvisible = false)

hm = heatmap!(ax1, I0range, U0range, modes,
                colormap = :Set1_3)
Colorbar(f[1, 2], hm, ticklabelsize = 35)
f